# Examples for Chapter 9

In [ ]:
import warnings
# these are innocuous but irritating
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
%matplotlib inline

In [ ]:
run auxil/subset -d [200,200,1000,1000] imagery/LT5_19980329.tif 

In [ ]:
run auxil/subset -d [200,200,1000,1000] imagery/LT5_19980516.tif 

In [ ]:
run scripts/dispms -f imagery/LT5_19980329_sub.tif -e 4 -p [4,5,7] \
-F imagery/LT5_19980516_sub.tif -E 4 -P [4,5,7] \
#-s '/home/mort/LaTeX/new projects/CRC4/Chapter9/fig9_1.eps'

## Naive methods

In [ ]:
import ee
import IPython.display as disp

ee.Initialize()

im1 = ee.Image('users/mortcanty/CRC4/Chapter9/LT5_19980329_sub')
im2 = ee.Image('users/mortcanty/CRC4/Chapter9/LT5_19980516_sub')
ndvi1 = im1.normalizedDifference(['b4', 'b3'])
ndvi2 = im2.normalizedDifference(['b4', 'b3'])
url = ndvi1.subtract(ndvi2) \
   .getThumbURL({'min':-0.3,'max':0.3})
disp.Image(url=url)

## Principal components

In [ ]:
import numpy as np
from osgeo import gdal
from osgeo.gdalconst import GA_ReadOnly
import matplotlib.pyplot as plt

gdal.AllRegister()
infile = 'imagery/LT5_19980329.tif'                 
inDataset = gdal.Open(infile,GA_ReadOnly)     
cols = inDataset.RasterXSize
rows = inDataset.RasterYSize    
band = inDataset.GetRasterBand(4)  
G1 = band.ReadAsArray(0,0,cols,rows).flatten()
infile = 'imagery/LT5_19980516.tif'                 
inDataset = gdal.Open(infile,GA_ReadOnly)       
band = inDataset.GetRasterBand(4)  
G2 = band.ReadAsArray(0,0,cols,rows).flatten()
idx = np.random.randint(0,rows*cols,10000)
p = plt.plot(G1[idx],G2[idx],'.')
#plt.savefig('/home/mort/LaTeX/new projects/CRC4/Chapter9/fig9_3.eps',bbox_inches='tight')

### Iterated PCA

In [ ]:
run scripts/ex9_1 imagery/LT5_19980329_sub.tif imagery/LT5_19980516_sub.tif

### Kernel PCA

In [ ]:
run scripts/dispms -f imagery/traffic1.jpg -p [1,2,3] -e 3 -F imagery/traffic2.jpg -P [1,2,3] -E 3 \
#-s '/home/mort/LaTeX/new projects/CRC4/Chapter9/fig9_6.eps'

In [ ]:
import numpy as np
from osgeo import gdal
from osgeo.gdalconst import GA_ReadOnly, GDT_Float32

gdal.AllRegister()
G = np.zeros((1000,1000,2))
inDataset = gdal.Open('imagery/traffic1.jpg',GA_ReadOnly)
G[:,:,0] = inDataset.GetRasterBand(1).ReadAsArray(0,0,1000,1000).astype(float)
inDataset = gdal.Open('imagery/traffic2.jpg',GA_ReadOnly)
G[:,:,1] = inDataset.GetRasterBand(1).ReadAsArray(0,0,1000,1000).astype(float)
driver = gdal.GetDriverByName('GTiff')
outDataset = driver.Create('imagery/traffic_bitemp.tif',1000,1000,2,GDT_Float32)
for k in range(2):        
    outBand = outDataset.GetRasterBand(k+1)
    outBand.WriteArray(G[:,:,k],0,0) 
    outBand.FlushCache() 
G[:,:,1] = G[:,:,1]**2  
outDataset = driver.Create('imagery/traffic_bitemp_nonlin.tif',1000,1000,2,GDT_Float32)
for k in range(2):        
    outBand = outDataset.GetRasterBand(k+1)
    outBand.WriteArray(G[:,:,k],0,0) 
    outBand.FlushCache() 
inDataset=None
outDataset=None

In [ ]:
run scripts/pca imagery/traffic_bitemp.tif

In [ ]:
run scripts/kpca  imagery/traffic_bitemp.tif 

In [ ]:
run scripts/dispms -f imagery/traffic_bitemp_pca.tif -e 2 -p [2,2,2] \
-F imagery/traffic_bitemp_kpca.tif -E 2 -P [2,2,2] \
#-s '/home/mort/LaTeX/new projects/CRC4/Chapter9/fig9_7.eps'

In [ ]:
run scripts/pca imagery/traffic_bitemp_nonlin.tif

In [ ]:
run scripts/kpca imagery/traffic_bitemp_nonlin.tif 

In [ ]:
run scripts/dispms -f imagery/traffic_bitemp_nonlin_pca.tif -e 2 -p [2,2,2] \
-F imagery/traffic_bitemp_nonlin_kpca.tif -E 2 -P [5,5,5] \
#-s '/home/mort/LaTeX/new projects/CRC4/Chapter9/fig9_8.eps'

## Multivariate Alteration Detection (MAD)

In [ ]:
# Run the iMAD transformation
%run scripts/iMad -i 50 -d [0,0,600,600] imagery/LT5_19980329_sub.tif \
                         imagery/LT5_19980516_sub.tif
 Set a significance level and calculate change map
%run scripts/iMadmap -m \
imagery/MAD(LT5_19980329_sub-LT5_19980516_sub).tif 0.0001

In [ ]:
run scripts/dispms -f imagery/MAD(LT5_19980329_sub-LT5_19980516_sub)_cmap.tif -e 3 -p [1,1,1] \
#-s '/home/mort/LaTeX/new projects/CRC4/Chapter9/fig9_9.eps'

In [ ]:
run scripts/dispms -f imagery/LE7_20010626 -e 3 -p [4,5,6] \
-F imagery/LE7_20010829 -E 3  -P [4,5,6] \
#-s '/home/mort/LaTeX/new projects/CRC4/Chapter9/fig9_11.eps'

In [ ]:
%run scripts/iMad imagery/LE7_20010626 imagery/LE7_20010829
%run scripts/iMadmap imagery/MAD(LE7_20010626-LE7_20010829) 0.0001 

In [ ]:
run scripts/dispms -f imagery/MAD(LE7_20010626-LE7_20010829) -e 2 -p [1,1,1] \
-F imagery/MAD(LE7_20010626-LE7_20010829) -E 2 -P [3,3,3] \
#-s '/home/mort/LaTeX/new projects/CRC4/Chapter9/fig9_13.eps'

In [ ]:
run scripts/dispms -f imagery/MAD(LE7_20010626-LE7_20010829)_cmap -e 3 -p [1,2,3] \
#-s '/home/mort/LaTeX/new projects/CRC4/Chapter9/fig9_14.eps'

In [ ]:
run scripts/em -K 4  imagery/MAD(LE7_20010626-LE7_20010829)

In [ ]:
%run scripts/dispms -f imagery/MAD(LE7_20010626-LE7_20010829)_em -c -d [400,0,200,200] \
-F imagery/LE7_20010829 -E 1 -D [400,0,200,200] -P [4,4,4] \
#-s '/home/mort/LaTeX/new projects/CRC4/Chapter9/fig9_15.eps'

## MAD on the Google Earth Engine

In [ ]:
import ee, math, time
from ipyleaflet import (Map,DrawControl,TileLayer)
from auxil.eeMad import imad
ee.Initialize()

def handle_draw(self, action, geo_json):
    global poly
    if action == 'created':
        coords =  geo_json['geometry']['coordinates']
        poly = ee.Geometry.Polygon(coords)
        
dc = DrawControl()
dc.on_draw(handle_draw)

def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

### Draw the map

In [ ]:
m = Map(center=[50.9, 6.4], zoom=11)
m.add_control(dc)

m

### iMAD wrapper function

In [ ]:
def iMad(cid,poly,sd1,ed1,sd2,ed2,bns,maxitr):
    collection = ee.ImageCollection(cid) \
         .filterBounds(poly) \
         .filterDate(ee.Date(sd1), ee.Date(ed1)) \
         .sort('system:time_start',False)
    image1 = ee.Image(collection.first()).select(bns)
    collection = ee.ImageCollection(cid) \
         .filterBounds(poly) \
         .filterDate(ee.Date(sd2), ee.Date(ed2)) \
         .sort('system:time_start',False)                   
    image2 = ee.Image(collection.first()).select(bns) 
    image2 = image2.register(image1,60)
    inputlist = ee.List.sequence(1,maxitr)
    first = ee.Dictionary({'done':ee.Number(0),
          'image':image1.addBands(image2).clip(poly),
          'allrhos': [ee.List.sequence(1,len(bns))],
          'chi2':ee.Image.constant(0),
          'MAD':ee.Image.constant(0)}) 
    madnames = ['MAD'+str(i+1) for i in range(len(bns))]
#  run the algorithm    
    result = ee.Dictionary(inputlist.iterate(imad,first))                
    MAD = ee.Image(result.get('MAD')).rename(madnames)
    return MAD

### Input data

In [ ]:
collectionid = 'LANDSAT/LE07/C01/T1_RT_TOA'
bandNames = ['B1','B2','B3','B4','B5','B7']
startDate1 = '2001-06-25'
endDate1 = '2001-06-27'
startDate2 = '2001-08-28'
endDate2 = '2001-08-30'
maxitr = 50
MAD = iMad(collectionid,poly,startDate1,
           endDate1,startDate2,endDate2,
           bandNames,maxitr)

### Display on map

In [ ]:
m.add_layer(
    TileLayer(url=GetTileLayerUrl(
        MAD.select('MAD1') \
        .visualize(min=-5, max=5)
    )
))

### Export to assets

In [ ]:
assexportname = 'users/mortcanty/imad/trail1'
assexport = ee.batch.Export.image.toAsset(MAD,
           description='assetExportTask', 
           assetId=assexportname,scale=30,maxPixels=1e9)
assexportid = str(assexport.id)
print '****Exporting to Assets, task id: %s'%assexportid
assexport.start() 

## Polarimetric SAR change detection

In [ ]:
!cat scripts/run_sar_seq.sh

In [ ]:
!scripts/run_sar_seqQ.sh S1A imagery/ 12 0.0001

In [ ]:
run scripts/dispms -f imagery/S1A_IW_SLC__1SDV_20141108T054351_20141108T054421_003186_003AB4_B367.tif -p [4,1,1] \
#-s '/home/mort/LaTeX/new projects/CRC4/Chapter9/fig9_17.eps'

In [ ]:
 run scripts/dispms -f  imagery/sarseqQ_cmap.tif -c -d [600,200,400,400] \
#-s '/home/mort/LaTeX/new projects/CRC4/Chapter9/fig9_18.eps'

In [ ]:
run scripts/dispms -f  imagery/sarseq_cmap.tif -c -d [600,200,400,400]

In [ ]:
!scripts/run_sar_seq.sh RS2 myimagery/ 12 0.0001

In [ ]:
run scripts/dispms -f myimagery/sarseq_cmap.tif -c -d [400,400,200,200] 

## Sequential SAR change detection on the GEE

In [ ]:
from auxil.eeSar_seq import run
run()

## Radiometric normalization

In [ ]:
run scripts/dispms  -f imagery/AST_20010409 -e 1 -p [1,2,3] \
-F imagery/AST_20050911 -E 1 -P [1,2,3] \
#-s '/home/mort/LaTeX/new projects/CRC4/Chapter9/fig9_24.eps'

In [ ]:
run scripts/iMad -p [1,2,3] imagery/AST_20010409 imagery/AST_20050911

In [ ]:
run scripts/radcal -p [1,2,3] imagery/MAD(AST_20010409-AST_20050911)

In [ ]:
run scripts/dispms  -f imagery/AST_20010409 -e 1 -p [1,2,3] \
-F imagery/AST_20050911_norm -E 1 -P [1,2,3] \
#-s '/home/mort/LaTeX/new projects/CRC4/Chapter9/fig9_25.eps'

### Code for Exercise 5

In [ ]:
import numpy as np
from osgeo import gdal
from osgeo.gdalconst import GA_ReadOnly, GDT_Float32

im1 = 'imagery/LE7_20010626'
im2 = 'imagery/LE7_20010829'
im2_toy = 'imagery/LE7_20010829_toy'
dim = 400
gdal.AllRegister()                 
inDataset1 = gdal.Open(im1,GA_ReadOnly) 
inDataset2 = gdal.Open(im2,GA_ReadOnly)
cols = inDataset1.RasterXSize
rows = inDataset1.RasterYSize 
bands = inDataset1.RasterCount
G1 = np.zeros((rows,cols,bands))
G2 = np.zeros((rows,cols,bands))
for k in range(bands):
    band = inDataset1.GetRasterBand(k+1)  
    G1[:,:,k] = band.ReadAsArray(0,0,cols,rows)
    band = inDataset2.GetRasterBand(k+1)  
    G2[:,:,k] = band.ReadAsArray(0,0,cols,rows)
G2[:dim,:dim,:] = G1[:dim,:dim,:] + \
               0.1*np.random.randn(dim,dim,bands)
driver = inDataset1.GetDriver()
outDataset = driver \
   .Create(im2_toy,cols,rows,bands,GDT_Float32)
for k in range(bands):        
            outBand = outDataset.GetRasterBand(k+1)
            outBand.WriteArray(G2[:,:,k],0,0) 
            outBand.FlushCache() 

In [ ]:
run scripts/dispms -f imagery/LE7_20010829 -e 3 -p [4,5,6] -F imagery/LE7_20010829_toy -E 3 -P [4,5,6] 